In [ ]:
# Donwload a resnet 18 from torch visin
import torchvision

model = torchvision.models.quantization.resnet18(pretrained=True )

# Display the model 

In [ ]:
from rich import print
print(model)

In [ ]:
# Display the fc layer weight distribution with seaborn
import seaborn as sns
import matplotlib.pyplot as plt
print(model.fc.weight.detach().numpy().flatten()[:25])
sns.displot(model.fc.weight.detach().numpy().flatten() ,  kind="ecdf")
sns.displot( data = model.fc.weight.detach().numpy().flatten() ,  
            kind="kde",
            rug = False,
            )
mean = model.fc.weight.detach().numpy().flatten().mean()
plt.title(f"Distribution of fc layer weights the mean is {mean} and the std is {model.fc.weight.detach().numpy().flatten().std():.3f}")
## Add a straight line at the mean
plt.axvline(model.fc.weight.detach().numpy().flatten().mean(), color='r')

In [ ]:
import lovely_tensors as lt
lt.monkey_patch()

In [ ]:
model.fc.weight.detach().plt()

In [ ]:
model.fc.weight.detach().flatten().plt()

# Apply torch.fx qat on resnet model

## Import all the Torch Quantization FX

In [ ]:
# Apply torch.fx qauntization aware traing resnet 18 model
from torch.fx import symbolic_trace
from torch.fx.graph_module import GraphModule
from torch.ao.quantization.quantize_fx import prepare_qat_fx, convert_fx

## Applied Prepare QAT FX 
1. Fusion
2. Model with QAT Modules 
3. Inserted Observers

### Create a shallow copy of model with copy

In [ ]:
import copy
fp32_model = copy.copy(model)

### Called the prepare_qat_fx from this PyTorch
- https://github.com/pytorch/pytorch/blob/6cef59487ab2e8a7124924f9a76cab381c638ac5/torch/ao/quantization/qconfig_mapping.py#L121
- https://github.com/pytorch/pytorch/tree/master/torch/ao/quantization/fx#12-qat-module-swap

In [ ]:
import torch
fp32_model.train()
## Creat the Qconfig Mapping
qconfig_mapping = torch.ao.quantization.qconfig_mapping.get_default_qat_qconfig_mapping("fbgemm")
print(qconfig_mapping)
## Called the prepare_qat_fx function
model = prepare_qat_fx(fp32_model, qconfig_mapping , example_inputs=torch.randn(1, 3, 224, 224))

In [ ]:
print(model)
assert isinstance(model, GraphModule)

In [ ]:
print( model.print_readable() ) 

### Called the Convert method in QAT

In [ ]:
from torch.ao.quantization.qconfig_mapping import get_default_qat_qconfig_mapping
print( qconfig_mapping.to_dict() )

In [ ]:
from torch.ao.quantization.fx.convert import convert
model_int8 = convert_fx(
    graph_module = model,
)

In [ ]:
from enum import Enum
class BackendType(Enum):
    Academic = 'Academic'
    Tensorrt = 'Tensorrt'
    SNPE = 'SNPE'
    PPLW8A16 = 'PPLW8A16'
    NNIE = 'NNIE'
    Vitis = 'Vitis'
    ONNX_QNN = 'ONNX_QNN'
    PPLCUDA = 'PPLCUDA'
    OPENVINO = 'OPENVINO'
    Tengine_u8 = "Tengine_u8"
    Tensorrt_NLP = "Tensorrt_NLP"
    Academic_NLP = "Academic_NLP"

In [ ]:
from rich import print
deploy_backend = BackendType.Academic
model_mode = 'Training' if model.training else 'Eval'
print("Quantize model Scheme: {} Mode: {}".format(deploy_backend, model_mode))

## Selec the Quantization Scheme of the model
- What is the Quantizatin of Activation 
- What is the Quantization of Weight
```
    BackendType.Tensorrt:   dict(qtype='affine',     # noqa: E241
                                 w_qscheme=QuantizeScheme(symmetry=True, per_channel=True, pot_scale=False, bit=8, symmetric_range=True),
                                 a_qscheme=QuantizeScheme(symmetry=True, per_channel=False, pot_scale=False, bit=8, symmetric_range=True),
                                 default_weight_quantize=LearnableFakeQuantize,
                                 default_act_quantize=LearnableFakeQuantize,
                                 default_weight_observer=MinMaxObserver,
                                 default_act_observer=EMAMinMaxObserver),
```

In [ ]:
# Get Qconfig
extra_qconfig_dict = prepare_custom_config_dict.get('extra_qconfig_dict', {})
qconfig = get_qconfig_by_platform(deploy_backend, extra_qconfig_dict)
